<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [5]:
using BenchmarkTools

┌ Info: Recompiling stale cache file /home/david/.julia/compiled/v1.1/BenchmarkTools/ZXPQo.ji for BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1184


In [204]:
function Log_Unnorm_Probs_B(x, W, Nh)
    @inline Log_1_plus_Exp(x) = x > 50.0 ? x : log(1.0 + exp(x));
    b     = view(W,:,1)
    ww    = view(W,:,2:Nh+1);
    x'*b .+ sum(Log_1_plus_Exp.(x'*ww),dims=2)
end;

In [205]:
Nsamples  = 1000
NNv       = 300;
NNh       = 600;
WW        = randn(NNv+1,NNh+1);
xxx       = rand([0.0,1.0],NNv+1,Nsamples)
xxx[1,:] .= 1.0;


In [209]:
size(Log_Unnorm_Probs_B(xxx, WW, NNh))

(1000, 1)

In [176]:
size(xxx' * view(WW,:,1))

(1000,)

In [179]:
size(xxx' * view(WW,:,1))

(1000,)

In [33]:
@time Log_Unnorm_Probs_B(xxx,WW,NNh);

  0.047082 seconds (19 allocations: 9.179 MiB)


In [13]:
@btime Log_Unnorm_Probs_B(xxx,WW,NNh);

  34.335 ms (15 allocations: 9.18 MiB)


In [273]:
function log_one_plus_exp(X::AbstractArray)
    Xnew = similar(X)
    
    for (i,x) in enumerate(X)
        if x > 50
            Xnew[i] = x
        else
             Xnew[i] =log(1.0 + exp(x))
        end
    end
    return Xnew
end

function log_one_plus_exp2(X::AbstractArray)
    Xnew = similar(X)
    
    for i in 1:length(X)
        if X[i] > 50
            Xnew[i] = X[i]
        else
            Xnew[i] =log(1.0 + exp(X[i]))
        end
    end
    return Xnew
end

function log_one_plus_exp_num(x)
    if x > 50 
        return x
    else
        return log(1.0 + exp(x))
    end
end


log_one_plus_exp_num (generic function with 1 method)

In [299]:
@inline function log_one_plus_exp_num(x)
    
    if x > 50.0 
        return x 
    else
        return log(1.0 + exp(x))
    end
end

function Log_Unnorm_Probs_B2(x, W, Nh)
    #b     = view(W,:,1)
    #ww    = view(W,:,2:Nh+1);
    
    b           = W[:,1]
    ww          = W[:,2:Nh+1]    
    propup      = x'*ww
    n_result    = size(x,2)
    result      = similar(1:n_result, Float64)
    propup_bias = x'*b 
    n_cols      = size(propup,2)
    
    @inbounds  for i in 1:length(result)
        sum_j = zero(eltype(result))
        @simd for j in 1:n_cols
            sum_j += log_one_plus_exp_num(propup[i,j])
        end
        result[i] = propup_bias[i] + sum_j
    end
    
    return result
    #return #x'*b .+ sum(log_one_plus_exp(x'*ww),dims=2)
end

function Log_Unnorm_Probs_B3(x, W, Nh)
    #b     = view(W,:,1)
    #ww    = view(W,:,2:Nh+1);
    
    b           = W[:,1]
    ww          = W[:,2:Nh+1]    
    propup      = x'*ww
    n_result    = size(x,2)
    result      = similar(1:n_result, Float64)
    propup_bias = x'*b 
    n_cols      = size(propup,2)
    
    @inbounds  for i in 1:length(result)
        sum_j = zero(eltype(result))
        @simd for j in 1:n_cols
            sum_j += exp(propup[i,j]) #log(1.0 + exp(propup[i,j]))
        end
        result[i] = propup_bias[i] + sum_j
    end
    
    return result
end

Log_Unnorm_Probs_B3 (generic function with 1 method)

In [297]:
@time Log_Unnorm_Probs_B2(xxx,WW,NNh);

  0.097515 seconds (85.69 k allocations: 10.372 MiB)


In [300]:
@btime Log_Unnorm_Probs_B3(xxx,WW,NNh);

  23.377 ms (7 allocations: 5.97 MiB)


In [305]:
exp(xxx)

DimensionMismatch: DimensionMismatch("matrix is not square: dimensions are (301, 1000)")

In [333]:
custom_exp(x) = 1 + x + x^2/factorial(2) + x^3/factorial(3) + x^4/factorial(4) + x^5/factorial(5) +  x^6/factorial(6) +  x^7/factorial(7)

custom_exp (generic function with 1 method)

In [334]:
exp(2)

7.38905609893065

In [340]:
custom_exp(2)

7.3809523809523805

In [347]:
@btime custom_exp(2)

  29.803 ns (0 allocations: 0 bytes)


7.3809523809523805

In [346]:
@btime exp(2)

  0.017 ns (0 allocations: 0 bytes)


7.38905609893065

In [339]:
custom_exp(0)

1.0

In [238]:
aux = Log_Unnorm_Probs_B2(xxx,WW,NNh);

In [244]:
aux2 = Log_Unnorm_Probs_B(xxx,WW,NNh);

In [270]:
@btime Log_Unnorm_Probs_B2(xxx,WW,NNh);

  40.632 ms (7 allocations: 5.97 MiB)


In [262]:
@btime Log_Unnorm_Probs_B(xxx,WW,NNh);

  34.444 ms (15 allocations: 9.18 MiB)


In [277]:
[x for x in 1:2:10]

5-element Array{Int64,1}:
 1
 3
 5
 7
 9

In [113]:
ww    = view(WW,:,2:NNh+1);
@btime sum(xxx' * ww, dims=2);

  6.766 ms (10 allocations: 4.59 MiB)


In [127]:
@btime sum(log_one_plus_exp.(xxx'*ww),dims=2);

  33.168 ms (14 allocations: 9.16 MiB)


In [156]:
@btime sum(log_one_plus_exp2(xxx'*ww),dims=2);

  34.367 ms (12 allocations: 9.16 MiB)


In [159]:
size(sum(log_one_plus_exp(xxx'*ww),dims=2))

(1000, 1)

In [190]:
size(xxx'* view(WW,:,1))

(1000,)

In [191]:
propup = xxx'*ww;

In [192]:
size(propup,2)

600